#### Genie

In [5]:
import datetime as dt
import pandas as pd
import time
import bs4
import requests

In [6]:
session = requests.Session()
session.headers.update({
    "Referer":"",
    "user-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
})

In [7]:
def transtime(x):
    return "".join(str(x).split(" ")[0].split("-"))

In [8]:
days = list(map(transtime,pd.date_range(pd.to_datetime('20230102'),dt.datetime.now(), freq='6D')))
page_dic={1:range(1,51),2:range(51,101)}
Genie = {}
Artists=[]

In [18]:
artist_url={}
for day in days:
    ranking_data=[]
    for page in page_dic:
        genie = session.get(f"https://www.genie.co.kr/chart/genre?ditc=W&ymd={day}&genrecode=M0100&pg={page}")
        time.sleep(2)
        soup = bs4.BeautifulSoup(genie.text)
        for n in page_dic[page]:
            data=soup.select(f".list.rank-{n}")[0]
            insert_data={}
            insert_data["ranking"]=data.select(".number")[0].text.split()[0]
            insert_data["title"]=data.select(".title.ellipsis")[0].text.strip()
            #아티스트 정보 삽입
            pre_data=data.select(".artist.ellipsis")
            insert_data["artist"]=pre_data[0].text.strip()
            url_num=pre_data[0].attrs["onclick"].split("(")[1].split(")")[0]
            artist_url[pre_data[0].text.strip()]='https://www.genie.co.kr/detail/artistInfo?xxnm='+url_num
            ranking_data.append(insert_data)
    Genie[day]=ranking_data

In [21]:
artist_url["(여자)아이들"]

'https://www.genie.co.kr/detail/artistInfo?xxnm=80632010'

#### 가수 정보

In [42]:
for artist in artist_url:
    insert_data={"name":artist}
    art=session.get(artist_url[artist])
    time.sleep(2)
    soup=bs4.BeautifulSoup(art.text)
    for data in soup.select(".info-data li"):
        if data.select(".attr img")[0].attrs["alt"]=="활동유형":
            insert_data["type"]=data.select(".value")[0].text.strip()
            if insert_data["type"].find("그룹")!=-1:
                insert_data["member"]=[]
                for member in soup.select(".artist-member-list li .name"):
                    insert_data["member"].append(member.text.strip())
                insert_data["member"]=",".join(insert_data["member"])
    Artists.append(insert_data)
        # if data.attrs["alt"]=="활동유형":
            # print(data.select(".value")[0].text.strip())

In [44]:
pd.DataFrame(Artists)

,name,type,member
0,NewJeans,여성/그룹,"민지 (MINJI),하니 (HANNI),다니엘 (DANIELLE),해린 (HAERI..."
1,윤하 (YOUNHA),여성/솔로,NaN
2,이영지,여성/솔로,NaN
3,LE SSERAFIM (르세라핌),여성/그룹,"미야와키 사쿠라,김채원,허윤진,카즈하,홍은채"
4,IVE (아이브),여성/그룹,"안유진 (IVE),가을 (IVE),레이 (IVE),장원영 (IVE),리즈 (IVE)..."
...,...,...,...
104,전상근,남성/솔로,NaN
105,Stray Kids (스트레이 키즈),남성/그룹,"방찬 (3RACHA),리노,창빈 (3RACHA),현진,한 (3RACHA),필릭스,승..."
106,제이세라 (J-Cera),여성/솔로,NaN
107,거미 (Gummy),여성/솔로,NaN
